# This generates a file containing the MJ background estimation from the ABCD method:
## A/B = C/D 
Where,
A = Signal Region (SR)
B = Signal Region Same Sign (SR-SS)
C = Control Region (CR)
D = Control Region Same Sign (CR-SS)

$$\frac{\text{SR}}{\text{SR}_{\text{SS}}} = \frac{\text{CR}}{\text{CR}_{\text{SS}}}$$


$$ \text{SR} = \text{SR}_{\text{SS}} \text{RQCD} $$

### Signal region path$
### Multi jet control region path
### Sample used for Z-QCDjets

In [13]:
import ROOT as r
from ROOT import gStyle
import numpy as np
import ctypes
import os
from IPython.display import display 
import pandas as pd 
from histogramHelpers import *


# Define the path to the channel and the directories for the regions.
channelPath = "/Users/diegomac/Documents/HEP/VBF-Analysis/TauTau/TauhadTaulep/High-Mass/"
SR = "DeterministicBDTCR/"
SRSS = "DeterministicBDTCRSS/"
CR = "NonIsoOS/"
CRSS = "NonIsoSS/"

# Select EWjj and QCDjj samples
EWjj = "Signal_Sherpa"
QCDjj = "Ztautau_SherpaRW"

RQCDwithCRs = False

# Remove previous MJ.root file
samples=[i for i in os.listdir(channelPath+SR) if ('.root' in i and 'Wjets' not in i)]
print(samples)
if "MJ.root" in samples:
    os.system("rm "+channelPath+SR+"MJ.root")

['MJ.root', 'Ztautau_SherpaRW.root', 'Ztautau_MGRW.root', 'Zjets.root', 'Signal_Sherpa.root', 'W_EWK_PoPy.root', 'Signal_PoPy.root', 'Data.root', 'MC.root', 'VV.root', 'Higgs.root', 'VV_EWK.root', 'W_EWK_Sherpa.root', 'ttbar.root', 'singletop.root']


In [14]:
import sys
sys.path.append('/Users/diegomac/Documents/HEP/VBF-Analysis/Scripts/')
from histogramHelpers import histogramsHighStatsZll,histogramsLowStatsZll,histogramsHighStatsZtautau,histogramsLowStatsZtautau
histos = histogramsVeryLowStatsZtautau

In [15]:
mcSamples = [EWjj,QCDjj]
backgroundSamples = ['VV',"ttbar",'singletop']#+['Wjets',]
if "Tau" in channelPath:
    backgroundSamples += ['Higgs','Zjets','W_EWK_Sherpa','VV_EWK']
mcSamples += backgroundSamples

# Open target file
multiJetFile =r.TFile.Open(channelPath+SR+"MJ.root", "RECREATE")

for histoName in histos:
    # Determine if histogram needs to be rebined
    rebinHistogram = False
    if len(histos[histoName])>2:
        rebinHistogram = True
    
    print(histoName,rebinHistogram)
    
    
    
    if RQCDwithCRs:
        # Calculate RQCD
        dataSubtractedHistoCR = dataSubtract(histoName,channelPath+CR,"Data",mcSamples,histos,rebin=rebinHistogram)
        dataSubtractedHistoCRSS = dataSubtract(histoName,channelPath+CRSS,"Data",mcSamples,histos,rebin=rebinHistogram)
        makeNegativeBinsZero(dataSubtractedHistoCR)
        makeNegativeBinsZero(dataSubtractedHistoCRSS)
        RQCD = dataSubtractedHistoCR.Integral(1,-1)/dataSubtractedHistoCRSS.Integral(1,-1)
    else:
        RQCD = 1.2
        UncerRQCD = 1.2*0.25
    
    # Calculate the MJ Background shape
    dataSubtractedHistoSRSS = dataSubtract(histoName,channelPath+SRSS,"Data",mcSamples,histos,rebin=rebinHistogram)
    dataSubtractedHistoSRSS.Scale(RQCD)
    makeNegativeBinsZero(dataSubtractedHistoSRSS)
    MJ = dataSubtractedHistoSRSS.Clone()
    scaleUncertainty(MJ,UncerRQCD/RQCD)
    
    # Save histogram in MJ.root file
    multiJetFile.WriteObject(MJ,histoName)
    
    
multiJetFile.Close()
    

n_bjets False
lepiso False
n_jets_interval False
flavourJet1_basic_all False
flavourJet2_basic_all False
elecPdgID_basic_all False
muonPdgID_basic_all False
tauPdgID_basic_all False
nLightJets_basic_all False
tau_pt True
lep_pt True
delta_phi True
delta_y True
omega True
rnn_score_1p True
rnn_score_3p True
ljet0_pt True
ljet1_pt True
pt_bal True
Z_centrality True
mass_jj True
reco_mass_i True
reco_mass_o True
reco_mass_ True
Z_pt_reco_i_basic_all True
Z_pt_reco_o_basic_all True
ratio_zpt_sumjetpt_basic_all True
vec_sum_pt_jets_basic_all True
moreCentralJet_basic_all True
lessCentralJet_basic_all True
delta_phijj_basic_all True
massTauClosestJet_basic_all True
massLepClosestJet_basic_all True
massTauFurthestJet_basic_all True
nuTauPt True
nuPtAssummetry_basic_all True
bdtScore True
lepNuPt True
pTsymmetry True
lepTransMass_basic_all True
tauTransMass_basic_all True
signedCentrality_basic_all True
visibleMass_basic_all True
recoVisibleMassRatio True


Warning in <TH1F::Rebin>: overflow entries will not be used when rebinning
Warning in <TH1F::Rebin>: overflow entries will not be used when rebinning
Warning in <TH1F::Rebin>: overflow entries will not be used when rebinning
Warning in <TH1F::Rebin>: overflow entries will not be used when rebinning
Warning in <TH1F::Rebin>: overflow entries will not be used when rebinning
Warning in <TH1F::Rebin>: Bin edge 2 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 3 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 5 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 6 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 2 of rebinned histogram does not match any